In [6]:
# -*- coding: utf-8 -*-
import logging
import numpy as np
from ase import units
from ase.build import bulk
from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator,AndersenIntegrator,NPTBerendsenIntegrator,NVTBerendsenIntegrator,VelocityVerletIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.utils.atoms_util import convert_atoms_to_upper
from matlantis_features.atoms import MatlantisAtoms
import pandas as pd
from pathlib import Path
from ase import units
from ase.build import add_vacuum
import re

# Matlantis / PFP
from matlantis_features.features.md import (
    MDFeature,
    ASEMDSystem,
    VelocityVerletIntegrator,
    LangevinIntegrator,
    NVTBerendsenIntegrator,
    NPTBerendsenIntegrator,
    NPTIntegrator,
)
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode

# --- ユーザー設定項目 (ここを編集してください) ---

# 1. 使用するIntegratorの種類を選択
#    選択肢: "NVE", "Langevin", "NVT_Berendsen", "NPT_Berendsen", "NPT"VelocityVerletIntegrator
# LangevinIntegrator
# AndersenIntegrator
# NVTBerendsenIntegrator
# NPTBerendsenIntegrator
# NPTIntegrator
INTEGRATOR_TYPE = "NVT_Berendsen"

# 2. 初期構造のファイルパスを指定 (ASEが読み込める形式ならOK)
#    例: "POSCAR", "input.cif", "structure.xyz"
#    指定しない場合は、Siのバルク結晶で計算します。
INPUT_STRUCTURE_FILE ="pvdf_opt2.traj"  # "your_structure_file.cif" など

# 3. MDシミュレーションの基本パラメータ
N_RUN_STEPS = 5000          # 実行するステップ数
TIME_STEP_FS = 1.0         # タイムステップ (fs)
TEMPERATURE_K = 1500        # 温度 (K)
PRESSURE_PA = 101325       # 圧力 (Pa) ※NPT系でのみ使用

# 4. ログとトラジェクトリの設定
SHOW_PROGRESS = True       # 進捗バーを表示するか
SHOW_LOGGER = True         # ログを標準出力に表示するか
LOGGER_INTERVAL = 50       # ログを出力する間隔 (ステップ数)
TRAJ_FILE_NAME = "NVT_1500K1.traj"  # 出力トラジェクトリファイル名
TRAJ_FREQ = 50             # トラジェクトリを保存する間隔 (ステップ数)
vacuum=0.2
# 5. PFPモデルの計算設定
PFP_MODEL_VERSION = 'v7.0.0'
PFP_CALC_MODE = EstimatorCalcMode.CRYSTAL_U0

# --- 設定項目ここまで ---


def select_integrator(integrator_type: str):
    """
    指定された文字列に応じて、対応するIntegratorオブジェクトを返す関数。
    """
    print(f"--- Integrator: '{integrator_type}' を選択しました ---")
    
    # NPT系で使用する圧力 (ASE unitsに変換)
    pressure_au = PRESSURE_PA * units.Pascal

    if integrator_type == "NVE":
        # Microcanonical (NVE) ensemble using the Velocity Verlet algorithm
        return VelocityVerletIntegrator(timestep=TIME_STEP_FS)
        
    elif integrator_type == "Langevin":
        # Langevin dynamics for NVT ensemble
        return LangevinIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            friction=0.002  # 摩擦係数 (調整が必要な場合あり)
        )

    elif integrator_type == "NVT_Berendsen":
        # NVT ensemble with Berendsen thermostat
        return NVTBerendsenIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            taut=100.0 * units.fs  # 温度緩和時間
        )
        
    elif integrator_type == "NPT_Berendsen":
        # NPT ensemble with Berendsen thermostat and barostat
        return NPTBerendsenIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            pressure=pressure_au,
            taut=100.0 * units.fs,    # 温度緩和時間
            taup=1000.0 * units.fs    # 圧力緩和時間
        )
        
    elif integrator_type == "NPT":
        # NPT ensemble with Nosé-Hoover thermostat and Parrinello-Rahman barostat
        return NPTIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            pressure=pressure_au,
            ttime=25.0 * units.fs,    # 温度緩和時間 (Nosé-Hoover)
            pfactor=75.0 * units.fs  # 圧力緩和時間 (Parrinello-Rahman)
        )
        
    else:
        raise ValueError(f"エラー: 指定されたIntegrator '{integrator_type}' は不正です。")


def run_md_simulation():
    """
    設定に基づいてMDシミュレーションを実行するメイン関数。
    """
    # Matlantis-featuresのロガー設定
    logger = logging.getLogger("matlantis_features")
    logger.setLevel(logging.INFO)

    # 1. PFP計算器の準備
    estimator_fn = pfp_estimator_fn(model_version=PFP_MODEL_VERSION, calc_mode=PFP_CALC_MODE)
    # MOLECULE = Atoms('H2O')                 # 追加する分子
    # SPACING = 3.0                           # 分子間隔 (Å)
    # Z_OFFSET = 3.0 
    # atoms = Trajectory(INPUT_STRUCTURE_FILE)[-1]
    # positions = atoms.get_positions()
    # center_x, center_y, _ = positions.mean(axis=0)
    # max_z = np.max(positions[:, 2])
    # grid_center = np.array([center_x, center_y, max_z + Z_OFFSET])
    
    # # 3. 9分子のグリッドを作成
    # molecule_grid = Atoms()
    # MOLECULE.center()
    # offset = (3 - 1) / 2.0  # 3x3グリッドの中心化オフセット
    
    # for ix in range(3):
    #     for iy in range(3):
    #         pos = grid_center + np.array([(ix - offset) * SPACING, (iy - offset) * SPACING, 0.0])
    #         mol_copy = MOLECULE.copy()
    #         mol_copy.translate(pos)
    #         molecule_grid.extend(mol_copy)
    
    # # 4. 既存構造とグリッドを結合
    # atoms.extend(molecule_grid)
    # vacuum = 0.5
            
    # # 3. 全原子の座標を取得し、各軸の最小値と最大値を計算
    # positions = atoms.get_positions()
    # min_coords = positions.min(axis=0)
    # max_coords = positions.max(axis=0)
    
    # # 4. 新しいセルの大きさを計算
    # #    (分子の最大幅) + (両側の真空層)
    # cell_lengths = (max_coords - min_coords) + 2 * vacuum
    # new_cell = np.diag(cell_lengths)
    
    # # 5. 計算したセルをatomsオブジェクトに設定
    # atoms.set_cell(new_cell)
    # atoms.pbc=True
    # atoms.center()
    # write("pvdf_h2o.xyz",atoms)
    # # 2. 初期構造の準備
    # if INPUT_STRUCTURE_FILE:
    #     try:
    #         existing_atoms = Trajectory(INPUT_STRUCTURE_FILE)[-1]
    #         from ase import Atoms
    #         h2o=Atoms('H2O')
    #         NUM_WATER_MOLECULES = 5
    #         # 既存構造と最初の水分子とのZ方向の距離 (Å)
    #         INITIAL_SEPARATION = 3.0
    #         # 水分子同士のZ方向の距離 (Å)
    #         WATER_SEPARATION = 3.0
    #         # 
    #         NUM_COLUMNS_NX=5
    #         WATER_SEPARATION_XY=3.0
    #         water_layer = Atoms()
    #         h2o.center(about=(0,0,0))
    #         for i in range(NUM_WATER_MOLECULES):
    #             # グリッド上の位置を計算 (ix: 列, iy: 行)
    #             ix = i % NUM_COLUMNS_NX
    #             iy = i // NUM_COLUMNS_NX
            
    #             # 配置する座標を計算
    #             position = np.array([ix * WATER_SEPARATION_XY, iy * WATER_SEPARATION_XY, 0.0])
            
    #             # テンプレートをコピーして、計算した位置に移動
    #             h2o_to_add = h2o.copy()
    #             h2o_to_add.translate(position)
            
    #             # レイヤーに水分子を追加
    #             water_layer.extend(h2o_to_add)
    #             print(f"{i+1}個目の水分子を (x, y) ≈ ({position[0]:.1f}, {position[1]:.1f}) に配置しました。")
            
    #         # レイヤー全体の重心が原点に来るように移動
    #         water_layer.center(about=(0,0,0))
    #         print("作成した水分子レイヤーを X-Y 平面でセンタリングしました。")


    #         # from pfcc_extras import show_gui, view_ngl
    #         # from ase.io import Trajectory,read
    #         max_z_existing = np.max(existing_atoms.get_positions()[:, 2])
    #         print(f"既存構造のZ方向の最大座標: {max_z_existing:.2f} Å")
    #         # 4. 水分子をZ方向に追加していく
    #         # 最初の水分子を配置するZ座標を計算
    #         current_z = max_z_existing + INITIAL_SEPARATION
    #         h2o_to_add.positions[:, 2] += current_z
    #         existing_atoms.extend(h2o_to_add)
            # for i in range(NUM_WATER_MOLECULES):
            #     # 配置する水分子のコピーを作成
            #     h2o_to_add = h2o.copy()
            
            #     # 水分子をZ方向に移動させる
            #     # h2o_to_add.positions は (N, 3) のNumpy配列なので、
            #     # Z座標の列 ([:, 2]) に current_z を加算する
            #     h2o_to_add.positions[:, 2] += current_z
            
            #     # 既存のAtomsオブジェクトに水分子の原子を追加
    #         #     existing_atoms.extend(h2o_to_add)
            
    #         #     print(f"{i+1}個目の水分子を Z ≈ {current_z:.2f} Å の位置に追加しました。")
            
    #         #     # 次の水分子を配置するZ座標を更新
    #         #     current_z += WATER_SEPARATION
    #         atoms=existing_atoms.copy()
    #         # estimator = Estimator(model_version=PFP_MODEL_VERSION,calc_mode=PFP_CALC_MODE)
    #         # calculator = ASECalculator(estimator)
    #         # atoms.calc=calculator
    #         # atoms.get_stress()
    #         vacuum = 0.5
            
    #         # 3. 全原子の座標を取得し、各軸の最小値と最大値を計算
    #         positions = atoms.get_positions()
    #         min_coords = positions.min(axis=0)
    #         max_coords = positions.max(axis=0)
            
    #         # 4. 新しいセルの大きさを計算
    #         #    (分子の最大幅) + (両側の真空層)
    #         cell_lengths = (max_coords - min_coords) + 2 * vacuum
    #         new_cell = np.diag(cell_lengths)
            
    #         # 5. 計算したセルをatomsオブジェクトに設定
    #         atoms.set_cell(new_cell)
    #         atoms.pbc=True
            
    #         print(f"構造ファイル '{INPUT_STRUCTURE_FILE}' を読み込みました。原子数: {len(atoms)}")
    #     except FileNotFoundError:
    #         print(f"エラー: ファイル '{INPUT_STRUCTURE_FILE}' が見つかりません。")
    #         return
    # else:
    #     atoms = bulk("Si", cubic=True) * (2, 2, 2)
    #     print("構造ファイルが指定されなかったため、Siのバルク構造を生成しました。")
    
    # 3. MDシステムの初期化
    atoms=read("pvdf_opt.xyz")
    positions = atoms.get_positions()
    min_coords = positions.min(axis=0)
    max_coords = positions.max(axis=0)
    
    # 4. 新しいセルの大きさを計算
    #    (分子の最大幅) + (両側の真空層)
    cell_lengths = (max_coords - min_coords) + 2 * vacuum
    new_cell = np.diag(cell_lengths)
    
    # 5. 計算したセルをatomsオブジェクトに設定
    atoms.set_cell(new_cell)
    atoms.pbc=True
    system = ASEMDSystem(atoms)
    
    # 乱数シードを固定して温度を初期化（再現性のため）
    system.init_temperature(temperature=TEMPERATURE_K, stationary=True, rng=np.random.RandomState(seed=12345))
    print(f"システムを {TEMPERATURE_K} K に初期化しました。")

    # 4. Integratorの選択と準備
    try:
        integrator = select_integrator(INTEGRATOR_TYPE)
    except ValueError as e:
        print(e)
        return

    # 5. MDFeatureの設定
    md = MDFeature(
        integrator=integrator,
        n_run=N_RUN_STEPS,
        show_progress_bar=SHOW_PROGRESS,
        show_logger=SHOW_LOGGER,
        logger_interval=LOGGER_INTERVAL,
        traj_file_name=TRAJ_FILE_NAME,
        traj_freq=TRAJ_FREQ,
        estimator_fn=estimator_fn,
    )

    # 6. MDシミュレーションの実行
    print(f"\n--- MDシミュレーションを開始します ({N_RUN_STEPS} ステップ) ---")
    md_results = md(system)

    print("\n--- MDシミュレーションが正常に完了しました ---")
    print(f"最終的なトラジェクトリは '{TRAJ_FILE_NAME}' に保存されました。")
    print(f"最終ステップ: {system.current_total_step}, 最終時間: {system.current_time:.2f} fs")


if __name__ == "__main__":
    run_md_simulation()

システムを 1500 K に初期化しました。
--- Integrator: 'NVT_Berendsen' を選択しました ---

--- MDシミュレーションを開始します (5000 ステップ) ---


  0%|          | 0/5000 [00:00<?, ?it/s]

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/NVT_1500K1.traj.
Note: The max disk size of /home/jovyan is about 98G.
steps:     0  energy：-4.45 eV/atom  total energy: -4.26 eV/atom  temperature: 1506.85 K  volume: 14620 Ang^3  density: 0.720 g/cm^3
steps:    50  energy：-4.33 eV/atom  total energy: -4.14 eV/atom  temperature: 1438.04 K  volume: 14620 Ang^3  density: 0.720 g/cm^3
steps:   100  energy：-4.34 eV/atom  total energy: -4.13 eV/atom  temperature: 1563.99 K  volume: 14620 Ang^3  density: 0.720 g/cm^3
steps:   150  energy：-4.32 eV/atom  total energy: -4.13 eV/atom  temperature: 1461.88 K  volume: 14620 Ang^3  density: 0.720 g/cm^3
steps:   200  energy：-4.34 eV/atom  total energy: -4.14 eV/atom  temperature: 1522.14 K  volume: 14620 Ang^3  density: 0.720 g/cm^3
steps:   250  energy：-4.33 eV/atom  total energy: -4.15 eV/atom  temperature: 1462.17 K  volume: 14620 Ang^3  density: 0.720 g/cm^3
steps:   300  energy：-4.34 eV/atom  total energy: -4.15 eV/atom 


--- MDシミュレーションが正常に完了しました ---
最終的なトラジェクトリは 'NVT_1500K1.traj' に保存されました。
最終ステップ: 5000, 最終時間: 5000.00 fs


In [1]:
## -*- coding: utf-8 -*-
import logging
import numpy as np
from ase import units
from ase.build import bulk
from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator,AndersenIntegrator,NPTBerendsenIntegrator,NVTBerendsenIntegrator,VelocityVerletIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.utils.atoms_util import convert_atoms_to_upper
from matlantis_features.atoms import MatlantisAtoms
import pandas as pd
from pathlib import Path
from ase import units
from ase.build import add_vacuum
import re

# Matlantis / PFP
from matlantis_features.features.md import (
    MDFeature,
    ASEMDSystem,
    VelocityVerletIntegrator,
    LangevinIntegrator,
    NVTBerendsenIntegrator,
    NPTBerendsenIntegrator,
    NPTIntegrator,
)
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode

# --- ユーザー設定項目 (ここを編集してください) ---

# 1. 使用するIntegratorの種類を選択
#    選択肢: "NVE", "Langevin", "NVT_Berendsen", "NPT_Berendsen", "NPT"VelocityVerletIntegrator
# LangevinIntegrator
# AndersenIntegrator
# NVTBerendsenIntegrator
# NPTBerendsenIntegrator
# NPTIntegrator
INTEGRATOR_TYPE = "NVT_Berendsen"

# 2. 初期構造のファイルパスを指定 (ASEが読み込める形式ならOK)
#    例: "POSCAR", "input.cif", "structure.xyz"
#    指定しない場合は、Siのバルク結晶で計算します。
# INPUT_STRUCTURE_FILE ="pvdf_opt2.traj"  # "your_structure_file.cif" など

# 3. MDシミュレーションの基本パラメータ
N_RUN_STEPS = 100000          # 実行するステップ数
TIME_STEP_FS = 1.0         # タイムステップ (fs)
TEMPERATURE_K = 600        # 温度 (K)
PRESSURE_PA = 101325       # 圧力 (Pa) ※NPT系でのみ使用
# --- 緩和計算の設定 ---
RELAX_TEMP = 300.0      # 緩和計算の温度 (K)
N_RELAX_STEPS = 5000    # 緩和計算のステップ数
RELAX_TRAJ_FILE_NAME = "relax_pvdf_water_random_600K.traj"
# 4. ログとトラジェクトリの設定
SHOW_PROGRESS = True       # 進捗バーを表示するか
SHOW_LOGGER = True         # ログを標準出力に表示するか
LOGGER_INTERVAL = 50       # ログを出力する間隔 (ステップ数)
TRAJ_FILE_NAME = "pvdf_water_random_600K.traj"  # 出力トラジェクトリファイル名
TRAJ_FREQ = 50             # トラジェクトリを保存する間隔 (ステップ数)
vacuum=0.2
# 5. PFPモデルの計算設定
PFP_MODEL_VERSION = 'v7.0.0'
PFP_CALC_MODE = EstimatorCalcMode.CRYSTAL_U0

# --- 設定項目ここまで ---


def select_integrator(integrator_type: str):
    """
    指定された文字列に応じて、対応するIntegratorオブジェクトを返す関数。
    """
    print(f"--- Integrator: '{integrator_type}' を選択しました ---")
    
    # NPT系で使用する圧力 (ASE unitsに変換)
    pressure_au = PRESSURE_PA * units.Pascal

    if integrator_type == "NVE":
        # Microcanonical (NVE) ensemble using the Velocity Verlet algorithm
        return VelocityVerletIntegrator(timestep=TIME_STEP_FS)
        
    elif integrator_type == "Langevin":
        # Langevin dynamics for NVT ensemble
        return LangevinIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            friction=0.002  # 摩擦係数 (調整が必要な場合あり)
        )

    elif integrator_type == "NVT_Berendsen":
        # NVT ensemble with Berendsen thermostat
        return NVTBerendsenIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            taut=100.0 * units.fs  # 温度緩和時間
        )
        
    elif integrator_type == "NPT_Berendsen":
        # NPT ensemble with Berendsen thermostat and barostat
        return NPTBerendsenIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            pressure=pressure_au,
            taut=100.0 * units.fs,    # 温度緩和時間
            taup=1000.0 * units.fs    # 圧力緩和時間
        )
        
    elif integrator_type == "NPT":
        # NPT ensemble with Nosé-Hoover thermostat and Parrinello-Rahman barostat
        return NPTIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            pressure=pressure_au,
            ttime=25.0 * units.fs,    # 温度緩和時間 (Nosé-Hoover)
            pfactor=75.0 * units.fs  # 圧力緩和時間 (Parrinello-Rahman)
        )
        
    else:
        raise ValueError(f"エラー: 指定されたIntegrator '{integrator_type}' は不正です。")


def run_md_simulation():
    """
    設定に基づいてMDシミュレーションを実行するメイン関数。
    """
    # Matlantis-featuresのロガー設定
    logger = logging.getLogger("matlantis_features")
    logger.setLevel(logging.INFO)

    # 1. PFP計算器の準備
    estimator_fn = pfp_estimator_fn(model_version=PFP_MODEL_VERSION, calc_mode=PFP_CALC_MODE)
    
    # 3. MDシステムの初期化
    atoms=read("pvdf_water_random.xyz")
    positions = atoms.get_positions()
    min_coords = positions.min(axis=0)
    max_coords = positions.max(axis=0)
    
    # 4. 新しいセルの大きさを計算
    #    (分子の最大幅) + (両側の真空層)
    cell_lengths = (max_coords - min_coords) + 2 * vacuum
    new_cell = np.diag(cell_lengths)
    
    # 5. 計算したセルをatomsオブジェクトに設定
    atoms.set_cell(new_cell)
    atoms.pbc=True
    system = ASEMDSystem(atoms)
    
    # 乱数シードを固定して温度を初期化（再現性のため）
    system.init_temperature(temperature=TEMPERATURE_K, stationary=True, rng=np.random.RandomState(seed=12345))
    print(f"システムを {TEMPERATURE_K} K に初期化しました。")
    integrator_relax=NVTBerendsenIntegrator(
            timestep=TIME_STEP_FS,
            temperature=300,
            taut=100.0 * units.fs  # 温度緩和時間
        )
    # 4. Integratorの選択と準備
    try:
        integrator = select_integrator(INTEGRATOR_TYPE)
    except ValueError as e:
        print(e)
        return
    # 5. MDFeatureの設定
    md_relax = MDFeature(
        integrator=integrator_relax,
        n_run=1000,
        show_progress_bar=SHOW_PROGRESS,
        show_logger=SHOW_LOGGER,
        logger_interval=LOGGER_INTERVAL,
        traj_file_name=RELAX_TRAJ_FILE_NAME,
        traj_freq=TRAJ_FREQ,
        estimator_fn=estimator_fn,
    )

    # 6. MDシミュレーションの実行
    print(f"\n--- MDシミュレーションを開始します ({N_RUN_STEPS} ステップ) ---")
    md_results = md_relax(system)
    # 5. MDFeatureの設定
    md = MDFeature(
        integrator=integrator,
        n_run=N_RUN_STEPS,
        show_progress_bar=SHOW_PROGRESS,
        show_logger=SHOW_LOGGER,
        logger_interval=LOGGER_INTERVAL,
        traj_file_name=TRAJ_FILE_NAME,
        traj_freq=TRAJ_FREQ,
        estimator_fn=estimator_fn,
    )

    # 6. MDシミュレーションの実行
    print(f"\n--- MDシミュレーションを開始します ({N_RUN_STEPS} ステップ) ---")
    md_results = md(system)

    print("\n--- MDシミュレーションが正常に完了しました ---")
    print(f"最終的なトラジェクトリは '{TRAJ_FILE_NAME}' に保存されました。")
    print(f"最終ステップ: {system.current_total_step}, 最終時間: {system.current_time:.2f} fs")


if __name__ == "__main__":
    run_md_simulation()

/home/jovyan/.py311/lib/python3.11/site-packages/pfp_api_client/__init__.py:42: UserWarning: New version of pfp-api-client is available. Please consider upgrading by `pip install -U pfp-api-client`.
  warnings.warn(


システムを 600 K に初期化しました。
--- Integrator: 'NVT_Berendsen' を選択しました ---

--- MDシミュレーションを開始します (100000 ステップ) ---


  0%|          | 0/1000 [00:00<?, ?it/s]

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/relax_pvdf_water_random_600K.traj.
Note: The max disk size of /home/jovyan is about 98G.
steps:     0  energy： 4.58 eV/atom  total energy:  4.65 eV/atom  temperature: 596.51 K  volume: 36313 Ang^3  density: 1.521 g/cm^3
steps:    50  energy：-0.88 eV/atom  total energy: 93829.55 eV/atom  temperature: 725904927.32 K  volume: 36313 Ang^3  density: 1.521 g/cm^3
steps:   100  energy：-1.53 eV/atom  total energy: 532.32 eV/atom  temperature: 4130086.99 K  volume: 36313 Ang^3  density: 1.521 g/cm^3
steps:   150  energy：-2.14 eV/atom  total energy: 40.26 eV/atom  temperature: 328060.70 K  volume: 36313 Ang^3  density: 1.521 g/cm^3


KeyboardInterrupt: 

In [3]:
## -*- coding: utf-8 -*-
import logging
import numpy as np
from ase import units
from ase.build import bulk
from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator,AndersenIntegrator,NPTBerendsenIntegrator,NVTBerendsenIntegrator,VelocityVerletIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.utils.atoms_util import convert_atoms_to_upper
from matlantis_features.atoms import MatlantisAtoms
import pandas as pd
from pathlib import Path
from ase import units
from ase.build import add_vacuum
import re

# Matlantis / PFP
from matlantis_features.features.md import (
    MDFeature,
    ASEMDSystem,
    VelocityVerletIntegrator,
    LangevinIntegrator,
    NVTBerendsenIntegrator,
    NPTBerendsenIntegrator,
    NPTIntegrator,
)
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode

# --- ユーザー設定項目 (ここを編集してください) ---

# 1. 使用するIntegratorの種類を選択
#    選択肢: "NVE", "Langevin", "NVT_Berendsen", "NPT_Berendsen", "NPT"VelocityVerletIntegrator
# LangevinIntegrator
# AndersenIntegrator
# NVTBerendsenIntegrator
# NPTBerendsenIntegrator
# NPTIntegrator
INTEGRATOR_TYPE = "NVT_Berendsen"

# 2. 初期構造のファイルパスを指定 (ASEが読み込める形式ならOK)
#    例: "POSCAR", "input.cif", "structure.xyz"
#    指定しない場合は、Siのバルク結晶で計算します。
# INPUT_STRUCTURE_FILE ="pvdf_opt2.traj"  # "your_structure_file.cif" など

# 3. MDシミュレーションの基本パラメータ
N_RUN_STEPS = 100000          # 実行するステップ数
TIME_STEP_FS = 1.0         # タイムステップ (fs)
TEMPERATURE_K = 600        # 温度 (K)
PRESSURE_PA = 101325       # 圧力 (Pa) ※NPT系でのみ使用
# --- 緩和計算の設定 ---
RELAX_TEMP = 300.0      # 緩和計算の温度 (K)
N_RELAX_STEPS = 5000    # 緩和計算のステップ数
RELAX_TRAJ_FILE_NAME = "relax_pvdf_water_random_600K.traj"
# 4. ログとトラジェクトリの設定
SHOW_PROGRESS = True       # 進捗バーを表示するか
SHOW_LOGGER = True         # ログを標準出力に表示するか
LOGGER_INTERVAL = 50       # ログを出力する間隔 (ステップ数)
TRAJ_FILE_NAME = "pvdf_water_random_600K.traj"  # 出力トラジェクトリファイル名
TRAJ_FREQ = 50             # トラジェクトリを保存する間隔 (ステップ数)
vacuum=0.2
# 5. PFPモデルの計算設定
PFP_MODEL_VERSION = 'v7.0.0'
PFP_CALC_MODE = EstimatorCalcMode.CRYSTAL_U0

# --- 設定項目ここまで ---


def select_integrator(integrator_type: str):
    """
    指定された文字列に応じて、対応するIntegratorオブジェクトを返す関数。
    """
    print(f"--- Integrator: '{integrator_type}' を選択しました ---")
    
    # NPT系で使用する圧力 (ASE unitsに変換)
    pressure_au = PRESSURE_PA * units.Pascal

    if integrator_type == "NVE":
        # Microcanonical (NVE) ensemble using the Velocity Verlet algorithm
        return VelocityVerletIntegrator(timestep=TIME_STEP_FS)
        
    elif integrator_type == "Langevin":
        # Langevin dynamics for NVT ensemble
        return LangevinIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            friction=0.002  # 摩擦係数 (調整が必要な場合あり)
        )

    elif integrator_type == "NVT_Berendsen":
        # NVT ensemble with Berendsen thermostat
        return NVTBerendsenIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            taut=100.0 * units.fs  # 温度緩和時間
        )
        
    elif integrator_type == "NPT_Berendsen":
        # NPT ensemble with Berendsen thermostat and barostat
        return NPTBerendsenIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            pressure=pressure_au,
            taut=100.0 * units.fs,    # 温度緩和時間
            taup=1000.0 * units.fs    # 圧力緩和時間
        )
        
    elif integrator_type == "NPT":
        # NPT ensemble with Nosé-Hoover thermostat and Parrinello-Rahman barostat
        return NPTIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            pressure=pressure_au,
            ttime=25.0 * units.fs,    # 温度緩和時間 (Nosé-Hoover)
            pfactor=75.0 * units.fs  # 圧力緩和時間 (Parrinello-Rahman)
        )
        
    else:
        raise ValueError(f"エラー: 指定されたIntegrator '{integrator_type}' は不正です。")


def run_md_simulation():
    """
    設定に基づいてMDシミュレーションを実行するメイン関数。
    """
    # Matlantis-featuresのロガー設定
    logger = logging.getLogger("matlantis_features")
    logger.setLevel(logging.INFO)

    # 1. PFP計算器の準備
    estimator_fn = pfp_estimator_fn(model_version=PFP_MODEL_VERSION, calc_mode=PFP_CALC_MODE)
    
    # 3. MDシステムの初期化
    atoms=Trajectory("pvdf_water_random_opt.traj")[-1]
    positions = atoms.get_positions()
    min_coords = positions.min(axis=0)
    max_coords = positions.max(axis=0)
    
    # 4. 新しいセルの大きさを計算
    #    (分子の最大幅) + (両側の真空層)
    cell_lengths = (max_coords - min_coords) + 2 * vacuum
    new_cell = np.diag(cell_lengths)
    
    # 5. 計算したセルをatomsオブジェクトに設定
    atoms.set_cell(new_cell)
    atoms.pbc=True
    system = ASEMDSystem(atoms)
    
    # 乱数シードを固定して温度を初期化（再現性のため）
    system.init_temperature(temperature=TEMPERATURE_K, stationary=True, rng=np.random.RandomState(seed=12345))
    print(f"システムを {TEMPERATURE_K} K に初期化しました。")
    integrator_relax=NVTBerendsenIntegrator(
            timestep=TIME_STEP_FS,
            temperature=300,
            taut=100.0 * units.fs  # 温度緩和時間
        )
    # 4. Integratorの選択と準備
    try:
        integrator = select_integrator(INTEGRATOR_TYPE)
    except ValueError as e:
        print(e)
        return
    # 5. MDFeatureの設定
    md_relax = MDFeature(
        integrator=integrator_relax,
        n_run=1000,
        show_progress_bar=SHOW_PROGRESS,
        show_logger=SHOW_LOGGER,
        logger_interval=LOGGER_INTERVAL,
        traj_file_name=RELAX_TRAJ_FILE_NAME,
        traj_freq=TRAJ_FREQ,
        estimator_fn=estimator_fn,
    )

    # 6. MDシミュレーションの実行
    print(f"\n--- MDシミュレーションを開始します ({N_RUN_STEPS} ステップ) ---")
    # md_results = md_relax(system)
    # 5. MDFeatureの設定
    md = MDFeature(
        integrator=integrator,
        n_run=N_RUN_STEPS,
        show_progress_bar=SHOW_PROGRESS,
        show_logger=SHOW_LOGGER,
        logger_interval=LOGGER_INTERVAL,
        traj_file_name=TRAJ_FILE_NAME,
        traj_freq=TRAJ_FREQ,
        estimator_fn=estimator_fn,
    )

    # 6. MDシミュレーションの実行
    print(f"\n--- MDシミュレーションを開始します ({N_RUN_STEPS} ステップ) ---")
    md_results = md(system)

    print("\n--- MDシミュレーションが正常に完了しました ---")
    print(f"最終的なトラジェクトリは '{TRAJ_FILE_NAME}' に保存されました。")
    print(f"最終ステップ: {system.current_total_step}, 最終時間: {system.current_time:.2f} fs")


if __name__ == "__main__":
    run_md_simulation()

システムを 600 K に初期化しました。
--- Integrator: 'NVT_Berendsen' を選択しました ---

--- MDシミュレーションを開始します (100000 ステップ) ---

--- MDシミュレーションを開始します (100000 ステップ) ---


  0%|          | 0/100000 [00:00<?, ?it/s]

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/pvdf_water_random_600K.traj.
Note: The max disk size of /home/jovyan is about 98G.
steps:     0  energy：-5.48 eV/atom  total energy: -5.40 eV/atom  temperature: 596.51 K  volume: 36464 Ang^3  density: 1.514 g/cm^3
steps:    50  energy：-2.12 eV/atom  total energy: 94.40 eV/atom  temperature: 746707.71 K  volume: 36464 Ang^3  density: 1.514 g/cm^3
steps:   100  energy：-2.67 eV/atom  total energy: 14.92 eV/atom  temperature: 136110.43 K  volume: 36464 Ang^3  density: 1.514 g/cm^3
steps:   150  energy：-3.10 eV/atom  total energy:  8.35 eV/atom  temperature: 88589.62 K  volume: 36464 Ang^3  density: 1.514 g/cm^3
steps:   200  energy：-3.34 eV/atom  total energy: -0.65 eV/atom  temperature: 20813.18 K  volume: 36464 Ang^3  density: 1.514 g/cm^3
steps:   250  energy：-3.47 eV/atom  total energy: -0.89 eV/atom  temperature: 19947.05 K  volume: 36464 Ang^3  density: 1.514 g/cm^3
steps:   300  energy：-3.58 eV/atom  total ener

KeyboardInterrupt: 

In [10]:
from pfcc_extras import show_gui, view_ngl
from ase.io import Trajectory,read,write

In [22]:
# list=["NVE", "Langevin", "NVT_Berendsen", "NPT_Berendsen", "NPT"]

# show_gui(Trajectory(f"md_trajectory_{list[2]}.traj"))
# show_gui(Trajectory("/home/jovyan/Kaori/MD/LiB_2/structure/NVT_1500K1.traj"))
# show_gui(read("pvdf441_water.xyz"))
# show_gui(Trajectory("pvdf_water_opt2.traj"))
show_gui(Trajectory("pvdf_big222_water_600K.traj"))

In [9]:
from pfcc_extras import show_gui, view_ngl
from ase.io import Trajectory,read,write
pvdf2=read("pvdf_editopt2.xyz") 
pvdf=pvdf2.copy()
positions=pvdf.get_positions()
cell=[positions[:,1].max(),positions[:,2].max(),positions[:,0].max()]
-positions[:,0].min()


4.64582949

In [4]:
# -*- coding: utf-8 -*-
import logging
import numpy as np
from ase import units
from ase.build import bulk
from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator,AndersenIntegrator,NPTBerendsenIntegrator,NVTBerendsenIntegrator,VelocityVerletIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.utils.atoms_util import convert_atoms_to_upper
from matlantis_features.atoms import MatlantisAtoms
import pandas as pd
from pathlib import Path
from ase import units
from ase.build import add_vacuum
import re

# Matlantis / PFP
from matlantis_features.features.md import (
    MDFeature,
    ASEMDSystem,
    VelocityVerletIntegrator,
    LangevinIntegrator,
    NVTBerendsenIntegrator,
    NPTBerendsenIntegrator,
    NPTIntegrator,
)
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode

# --- ユーザー設定項目 (ここを編集してください) ---

# 1. 使用するIntegratorの種類を選択
#    選択肢: "NVE", "Langevin", "NVT_Berendsen", "NPT_Berendsen", "NPT"VelocityVerletIntegrator
# LangevinIntegrator
# AndersenIntegrator
# NVTBerendsenIntegrator
# NPTBerendsenIntegrator
# NPTIntegrator
INTEGRATOR_TYPE = "NVT_Berendsen"

# 2. 初期構造のファイルパスを指定 (ASEが読み込める形式ならOK)
#    例: "POSCAR", "input.cif", "structure.xyz"
#    指定しない場合は、Siのバルク結晶で計算します。
INPUT_STRUCTURE_FILE ="pvdf_opt2.traj"  # "your_structure_file.cif" など

# 3. MDシミュレーションの基本パラメータ
N_RUN_STEPS = 5000          # 実行するステップ数
TIME_STEP_FS = 1.0         # タイムステップ (fs)
TEMPERATURE_K = 2000        # 温度 (K)
PRESSURE_PA = 101325       # 圧力 (Pa) ※NPT系でのみ使用

# 4. ログとトラジェクトリの設定
SHOW_PROGRESS = True       # 進捗バーを表示するか
SHOW_LOGGER = True         # ログを標準出力に表示するか
LOGGER_INTERVAL = 50       # ログを出力する間隔 (ステップ数)
TRAJ_FILE_NAME = "NVT_2000K.traj"  # 出力トラジェクトリファイル名
TRAJ_FREQ = 50             # トラジェクトリを保存する間隔 (ステップ数)
vacuum=0.2
# 5. PFPモデルの計算設定
PFP_MODEL_VERSION = 'v7.0.0'
PFP_CALC_MODE = EstimatorCalcMode.CRYSTAL_U0

# --- 設定項目ここまで ---


def select_integrator(integrator_type: str):
    """
    指定された文字列に応じて、対応するIntegratorオブジェクトを返す関数。
    """
    print(f"--- Integrator: '{integrator_type}' を選択しました ---")
    
    # NPT系で使用する圧力 (ASE unitsに変換)
    pressure_au = PRESSURE_PA * units.Pascal

    if integrator_type == "NVE":
        # Microcanonical (NVE) ensemble using the Velocity Verlet algorithm
        return VelocityVerletIntegrator(timestep=TIME_STEP_FS)
        
    elif integrator_type == "Langevin":
        # Langevin dynamics for NVT ensemble
        return LangevinIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            friction=0.002  # 摩擦係数 (調整が必要な場合あり)
        )

    elif integrator_type == "NVT_Berendsen":
        # NVT ensemble with Berendsen thermostat
        return NVTBerendsenIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            taut=100.0 * units.fs  # 温度緩和時間
        )
        
    elif integrator_type == "NPT_Berendsen":
        # NPT ensemble with Berendsen thermostat and barostat
        return NPTBerendsenIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            pressure=pressure_au,
            taut=100.0 * units.fs,    # 温度緩和時間
            taup=1000.0 * units.fs    # 圧力緩和時間
        )
        
    elif integrator_type == "NPT":
        # NPT ensemble with Nosé-Hoover thermostat and Parrinello-Rahman barostat
        return NPTIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            pressure=pressure_au,
            ttime=25.0 * units.fs,    # 温度緩和時間 (Nosé-Hoover)
            pfactor=75.0 * units.fs  # 圧力緩和時間 (Parrinello-Rahman)
        )
        
    else:
        raise ValueError(f"エラー: 指定されたIntegrator '{integrator_type}' は不正です。")


def run_md_simulation():
    """
    設定に基づいてMDシミュレーションを実行するメイン関数。
    """
    # Matlantis-featuresのロガー設定
    logger = logging.getLogger("matlantis_features")
    logger.setLevel(logging.INFO)

    # 1. PFP計算器の準備
    estimator_fn = pfp_estimator_fn(model_version=PFP_MODEL_VERSION, calc_mode=PFP_CALC_MODE)
    # MOLECULE = Atoms('H2O')                 # 追加する分子
    # SPACING = 3.0                           # 分子間隔 (Å)
    # Z_OFFSET = 3.0 
    # atoms = Trajectory(INPUT_STRUCTURE_FILE)[-1]
    # positions = atoms.get_positions()
    # center_x, center_y, _ = positions.mean(axis=0)
    # max_z = np.max(positions[:, 2])
    # grid_center = np.array([center_x, center_y, max_z + Z_OFFSET])
    
    # # 3. 9分子のグリッドを作成
    # molecule_grid = Atoms()
    # MOLECULE.center()
    # offset = (3 - 1) / 2.0  # 3x3グリッドの中心化オフセット
    
    # for ix in range(3):
    #     for iy in range(3):
    #         pos = grid_center + np.array([(ix - offset) * SPACING, (iy - offset) * SPACING, 0.0])
    #         mol_copy = MOLECULE.copy()
    #         mol_copy.translate(pos)
    #         molecule_grid.extend(mol_copy)
    
    # # 4. 既存構造とグリッドを結合
    # atoms.extend(molecule_grid)
    # vacuum = 0.5
            
    # # 3. 全原子の座標を取得し、各軸の最小値と最大値を計算
    # positions = atoms.get_positions()
    # min_coords = positions.min(axis=0)
    # max_coords = positions.max(axis=0)
    
    # # 4. 新しいセルの大きさを計算
    # #    (分子の最大幅) + (両側の真空層)
    # cell_lengths = (max_coords - min_coords) + 2 * vacuum
    # new_cell = np.diag(cell_lengths)
    
    # # 5. 計算したセルをatomsオブジェクトに設定
    # atoms.set_cell(new_cell)
    # atoms.pbc=True
    # atoms.center()
    # write("pvdf_h2o.xyz",atoms)
    # # 2. 初期構造の準備
    # if INPUT_STRUCTURE_FILE:
    #     try:
    #         existing_atoms = Trajectory(INPUT_STRUCTURE_FILE)[-1]
    #         from ase import Atoms
    #         h2o=Atoms('H2O')
    #         NUM_WATER_MOLECULES = 5
    #         # 既存構造と最初の水分子とのZ方向の距離 (Å)
    #         INITIAL_SEPARATION = 3.0
    #         # 水分子同士のZ方向の距離 (Å)
    #         WATER_SEPARATION = 3.0
    #         # 
    #         NUM_COLUMNS_NX=5
    #         WATER_SEPARATION_XY=3.0
    #         water_layer = Atoms()
    #         h2o.center(about=(0,0,0))
    #         for i in range(NUM_WATER_MOLECULES):
    #             # グリッド上の位置を計算 (ix: 列, iy: 行)
    #             ix = i % NUM_COLUMNS_NX
    #             iy = i // NUM_COLUMNS_NX
            
    #             # 配置する座標を計算
    #             position = np.array([ix * WATER_SEPARATION_XY, iy * WATER_SEPARATION_XY, 0.0])
            
    #             # テンプレートをコピーして、計算した位置に移動
    #             h2o_to_add = h2o.copy()
    #             h2o_to_add.translate(position)
            
    #             # レイヤーに水分子を追加
    #             water_layer.extend(h2o_to_add)
    #             print(f"{i+1}個目の水分子を (x, y) ≈ ({position[0]:.1f}, {position[1]:.1f}) に配置しました。")
            
    #         # レイヤー全体の重心が原点に来るように移動
    #         water_layer.center(about=(0,0,0))
    #         print("作成した水分子レイヤーを X-Y 平面でセンタリングしました。")


    #         # from pfcc_extras import show_gui, view_ngl
    #         # from ase.io import Trajectory,read
    #         max_z_existing = np.max(existing_atoms.get_positions()[:, 2])
    #         print(f"既存構造のZ方向の最大座標: {max_z_existing:.2f} Å")
    #         # 4. 水分子をZ方向に追加していく
    #         # 最初の水分子を配置するZ座標を計算
    #         current_z = max_z_existing + INITIAL_SEPARATION
    #         h2o_to_add.positions[:, 2] += current_z
    #         existing_atoms.extend(h2o_to_add)
            # for i in range(NUM_WATER_MOLECULES):
            #     # 配置する水分子のコピーを作成
            #     h2o_to_add = h2o.copy()
            
            #     # 水分子をZ方向に移動させる
            #     # h2o_to_add.positions は (N, 3) のNumpy配列なので、
            #     # Z座標の列 ([:, 2]) に current_z を加算する
            #     h2o_to_add.positions[:, 2] += current_z
            
            #     # 既存のAtomsオブジェクトに水分子の原子を追加
    #         #     existing_atoms.extend(h2o_to_add)
            
    #         #     print(f"{i+1}個目の水分子を Z ≈ {current_z:.2f} Å の位置に追加しました。")
            
    #         #     # 次の水分子を配置するZ座標を更新
    #         #     current_z += WATER_SEPARATION
    #         atoms=existing_atoms.copy()
    #         # estimator = Estimator(model_version=PFP_MODEL_VERSION,calc_mode=PFP_CALC_MODE)
    #         # calculator = ASECalculator(estimator)
    #         # atoms.calc=calculator
    #         # atoms.get_stress()
    #         vacuum = 0.5
            
    #         # 3. 全原子の座標を取得し、各軸の最小値と最大値を計算
    #         positions = atoms.get_positions()
    #         min_coords = positions.min(axis=0)
    #         max_coords = positions.max(axis=0)
            
    #         # 4. 新しいセルの大きさを計算
    #         #    (分子の最大幅) + (両側の真空層)
    #         cell_lengths = (max_coords - min_coords) + 2 * vacuum
    #         new_cell = np.diag(cell_lengths)
            
    #         # 5. 計算したセルをatomsオブジェクトに設定
    #         atoms.set_cell(new_cell)
    #         atoms.pbc=True
            
    #         print(f"構造ファイル '{INPUT_STRUCTURE_FILE}' を読み込みました。原子数: {len(atoms)}")
    #     except FileNotFoundError:
    #         print(f"エラー: ファイル '{INPUT_STRUCTURE_FILE}' が見つかりません。")
    #         return
    # else:
    #     atoms = bulk("Si", cubic=True) * (2, 2, 2)
    #     print("構造ファイルが指定されなかったため、Siのバルク構造を生成しました。")
    
    # 3. MDシステムの初期化
    atoms=read("pvdf_opt.xyz")
    positions = atoms.get_positions()
    min_coords = positions.min(axis=0)
    max_coords = positions.max(axis=0)
    
    # 4. 新しいセルの大きさを計算
    #    (分子の最大幅) + (両側の真空層)
    cell_lengths = (max_coords - min_coords) + 2 * vacuum
    new_cell = np.diag(cell_lengths)
    
    # 5. 計算したセルをatomsオブジェクトに設定
    atoms.set_cell(new_cell)
    atoms.pbc=True
    system = ASEMDSystem(atoms)
    
    # 乱数シードを固定して温度を初期化（再現性のため）
    system.init_temperature(temperature=TEMPERATURE_K, stationary=True, rng=np.random.RandomState(seed=12345))
    print(f"システムを {TEMPERATURE_K} K に初期化しました。")

    # 4. Integratorの選択と準備
    try:
        integrator = select_integrator(INTEGRATOR_TYPE)
    except ValueError as e:
        print(e)
        return

    # 5. MDFeatureの設定
    md = MDFeature(
        integrator=integrator,
        n_run=N_RUN_STEPS,
        show_progress_bar=SHOW_PROGRESS,
        show_logger=SHOW_LOGGER,
        logger_interval=LOGGER_INTERVAL,
        traj_file_name=TRAJ_FILE_NAME,
        traj_freq=TRAJ_FREQ,
        estimator_fn=estimator_fn,
    )

    # 6. MDシミュレーションの実行
    print(f"\n--- MDシミュレーションを開始します ({N_RUN_STEPS} ステップ) ---")
    md_results = md(system)

    print("\n--- MDシミュレーションが正常に完了しました ---")
    print(f"最終的なトラジェクトリは '{TRAJ_FILE_NAME}' に保存されました。")
    print(f"最終ステップ: {system.current_total_step}, 最終時間: {system.current_time:.2f} fs")


if __name__ == "__main__":
    run_md_simulation()

システムを 2000 K に初期化しました。
--- Integrator: 'NVT_Berendsen' を選択しました ---

--- MDシミュレーションを開始します (5000 ステップ) ---


  0%|          | 0/5000 [00:00<?, ?it/s]

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/NVT_2000K.traj.
Note: The max disk size of /home/jovyan is about 98G.
steps:     0  energy：-4.45 eV/atom  total energy: -4.19 eV/atom  temperature: 2009.13 K  volume: 14620 Ang^3  density: 0.720 g/cm^3
steps:    50  energy：-4.27 eV/atom  total energy: -4.03 eV/atom  temperature: 1895.69 K  volume: 14620 Ang^3  density: 0.720 g/cm^3
steps:   100  energy：-4.27 eV/atom  total energy: -4.02 eV/atom  temperature: 1959.39 K  volume: 14620 Ang^3  density: 0.720 g/cm^3
steps:   150  energy：-4.28 eV/atom  total energy: -4.02 eV/atom  temperature: 2028.22 K  volume: 14620 Ang^3  density: 0.720 g/cm^3
steps:   200  energy：-4.28 eV/atom  total energy: -4.02 eV/atom  temperature: 2035.42 K  volume: 14620 Ang^3  density: 0.720 g/cm^3
steps:   250  energy：-4.27 eV/atom  total energy: -4.02 eV/atom  temperature: 1959.59 K  volume: 14620 Ang^3  density: 0.720 g/cm^3
steps:   300  energy：-4.28 eV/atom  total energy: -4.02 eV/atom  


--- MDシミュレーションが正常に完了しました ---
最終的なトラジェクトリは 'NVT_2000K.traj' に保存されました。
最終ステップ: 5000, 最終時間: 5000.00 fs


In [64]:
MOLECULE = Atoms('H2O')                 # 追加する分子
SPACING = 3.0                           # 分子間隔 (Å)
Z_OFFSET = 3.0 
atoms = Trajectory(INPUT_STRUCTURE_FILE)[-1]
positions = atoms.get_positions()
center_x, center_y, _ = positions.mean(axis=0)
max_z = np.max(positions[:, 2])
grid_center = np.array([center_x, center_y, max_z + Z_OFFSET])
print(atoms.get_chemical_symbols())
# 3. 9分子のグリッドを作成
molecule_grid = Atoms()
MOLECULE.center()
offset = (3 - 1) / 2.0  # 3x3グリッドの中心化オフセット

for ix in range(3):
    for iy in range(3):
        pos = grid_center + np.array([(ix - offset) * SPACING, (iy - offset) * SPACING, 0.0])
        mol_copy = MOLECULE.copy()
        mol_copy.translate(pos)
        molecule_grid=molecule_grid+mol_copy

# 4. 既存構造とグリッドを結合
atoms=atoms+molecule_grid
vacuum = 0.5
        
# 3. 全原子の座標を取得し、各軸の最小値と最大値を計算
positions = atoms.get_positions()
min_coords = positions.min(axis=0)
max_coords = positions.max(axis=0)
print(atoms.get_chemical_symbols())
# 4. 新しいセルの大きさを計算
#    (分子の最大幅) + (両側の真空層)
cell_lengths = (max_coords - min_coords) + 2 * vacuum
new_cell = np.diag(cell_lengths)

# 5. 計算したセルをatomsオブジェクトに設定
atoms.set_cell(new_cell)
atoms.pbc=True
atoms.center()
write("pvdf_h2o.xyz",atoms)

['C', 'C', 'F', 'F', 'C', 'C', 'C', 'C', 'H', 'H', 'F', 'F', 'H', 'H', 'F', 'F', 'C', 'C', 'C', 'C', 'H', 'H', 'F', 'F', 'H', 'H', 'F', 'F', 'C', 'C', 'C', 'C', 'H', 'H', 'F', 'F', 'H', 'H', 'F', 'F', 'C', 'C', 'C', 'C', 'H', 'H', 'F', 'F', 'H', 'H', 'F', 'C', 'C', 'H', 'H', 'H', 'H', 'F', 'F', 'C', 'C', 'F', 'F', 'C', 'C', 'C', 'C', 'H', 'H', 'F', 'H', 'H', 'F', 'F', 'C', 'C', 'C', 'C', 'H', 'H', 'F', 'F', 'H', 'H', 'F', 'F', 'C', 'C', 'C', 'C', 'H', 'H', 'F', 'F', 'H', 'H', 'F', 'F', 'C', 'C', 'H', 'H', 'H', 'H', 'F', 'F', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'F', 'F', 'H', 'H', 'F', 'F', 'C', 'C', 'C', 'C', 'H', 'H', 'F', 'F', 'H', 'H', 'F', 'F', 'C', 'C', 'C', 'C', 'H', 'H', 'F', 'F', 'H', 'F', 'F', 'C', 'C', 'C', 'H', 'H', 'F', 'F', 'F', 'H', 'H', 'C', 'C', 'F', 'F', 'C', 'C', 'C', 'C', 'H', 'H', 'F', 'F', 'H', 'H', 'F', 'F', 'C', 'C', 'C', 'C', 'H', 'H', 'F', 'F', 'H', 'H', 'F', 'F', 'C', 'C', 'C', 'C', 'H', 'H', 'F', 'F', 'H', 'H', 'H', 'H', 'F', 'F', 'H', 'C', 'C', 'F', 'F', 'C',

In [46]:
from ase import Atoms
h2o=Atoms('H2O')
from pfcc_extras import show_gui, view_ngl
from ase.io import Trajectory,read
show_gui(h2o)

In [41]:
help(atoms)

Help on Atoms in module ase.atoms object:

class Atoms(builtins.object)
 |  Atoms(symbols=None, positions=None, numbers=None, tags=None, momenta=None, masses=None, magmoms=None, charges=None, scaled_positions=None, cell=None, pbc=None, celldisp=None, constraint=None, calculator=None, info=None, velocities=None)
 |  
 |  Atoms object.
 |  
 |  The Atoms object can represent an isolated molecule, or a
 |  periodically repeated structure.  It has a unit cell and
 |  there may be periodic boundary conditions along any of the three
 |  unit cell axes.
 |  Information about the atoms (atomic numbers and position) is
 |  stored in ndarrays.  Optionally, there can be information about
 |  tags, momenta, masses, magnetic moments and charges.
 |  
 |  In order to calculate energies, forces and stresses, a calculator
 |  object has to attached to the atoms object.
 |  
 |  Parameters
 |  ----------
 |  symbols : str | list[str] | list[Atom]
 |      Chemical formula, a list of chemical symbols, or

In [35]:
# write('data.lammps', atoms, format='lammps-data', atom_style='atomic')
help(write)

Help on function write in module ase.io.formats:

write(filename: Union[str, pathlib.PurePath, IO], images: Union[ase.atoms.Atoms, Sequence[ase.atoms.Atoms]], format: str = None, parallel: bool = True, append: bool = False, **kwargs: Any) -> None
    Write Atoms object(s) to file.
    
    filename: str or file
        Name of the file to write to or a file descriptor.  The name '-'
        means standard output.
    images: Atoms object or list of Atoms objects
        A single Atoms object or a list of Atoms objects.
    format: str
        Used to specify the file-format.  If not given, the
        file-format will be taken from suffix of the filename.
    parallel: bool
        Default is to write on master only.  Use parallel=False to write
        from all slaves.
    append: bool
        Default is to open files in 'w' or 'wb' mode, overwriting
        existing files.  In some cases opening the file in 'a' or 'ab'
        mode (appending) is useful,
        e.g. writing trajecto

In [38]:
import ase.io
from ase.io.lammpsdata import write_lammps_data # lammpsdataモジュールから直接インポート

# --- ユーザー設定項目 ---
XYZ_FILE = "pvdf_opt2.traj"
DATA_FILE = "data.pvdf"
INPUT_FILE = "in.pvdf_pyrolysis"
# VACUUM_LAYER = .0

# --- 1. XYZファイルを読み込み、LAMMPSデータファイルを作成 ---
try:
    atoms = Trajectory(XYZ_FILE)[-1]
    # atoms.center(vacuum=VACUUM_LAYER)
    atoms.pbc = True

    # write_lammps_data を直接使用
    print(f"writing lammps data to {DATA_FILE}")
    with open(DATA_FILE, 'w') as fd:
        write_lammps_data(fd, atoms, atom_style='full') # ファイルオブジェクト(fd)を渡す

    print(f"✅ LAMMPSデータファイル '{DATA_FILE}' を作成しました。")

except FileNotFoundError:
    print(f"❌ エラー: 入力ファイル '{XYZ_FILE}' が見つかりません。")
    exit()

# # --- 2. LAMMPSのインプットスクリプトを作成 (この部分は変更なし) ---
# lammps_input_script = f"""
# # LAMMPSインプットスクリプト: PVDFの熱分解シミュレーション
# # 生成元: ASE (Atomic Simulation Environment)

# # --- 1. 初期化 ---
# units          real
# boundary       p p p
# atom_style     full

# # --- 2. 構造読み込み ---
# read_data      {DATA_FILE}

# # (以下、前回のスクリプトと同様...)
# # --- 3. 力場設定 (ReaxFF) ---
# pair_style     reaxff NULL checkqeq no
# pair_coeff     * * ffield.reax.CHON C H F
# fix            1 all qeq/reaxff 1 0.0 10.0 1.0e-6 reaxff

# # --- 4. エネルギー最小化 ---
# minimize       1.0e-4 1.0e-6 1000 10000

# # --- 5. アニーリング (200K -> 500K -> 200Kを10サイクル) ---
# timestep       0.2
# thermo         1000
# print "========= Starting Annealing Loop ========="
# variable       i loop 10
# label          loop_anneal
#   fix          anneal all nvt temp 200.0 500.0 100.0
#   run          5000
#   unfix        anneal
#   fix          anneal all nvt temp 500.0 200.0 100.0
#   run          5000
#   unfix        anneal
#   next         i
# jump           SELF loop_anneal
# print "========= Annealing Finished ========="

# # --- 6. 昇温プロセス ---
# reset_timestep 0
# thermo         5000
# fix            heating all nvt temp 200.0 3000.0 100.0
# dump           heating_dump all custom 5000 dump.heating.*.lammpstrj id type x y z
# run            100000
# unfix          heating
# undump         heating_dump

# # --- 7. 定温プロセス (NVT) ---
# reset_timestep 0
# thermo         10000
# fix            isothermal all nvt temp 3000.0 3000.0 100.0
# dump           iso_dump all custom 10000 dump.isothermal.*.lammpstrj id type x y z
# run            30000000

# print "========= Simulation Finished ========="
# """

# with open(INPUT_FILE, 'w') as f:
#     f.write(lammps_input_script)

print(f"✅ LAMMPSインプットスクリプト '{INPUT_FILE}' を作成しました。")

writing lammps data to data.pvdf


LinAlgError: Singular matrix

In [29]:
# -*- coding: utf-8 -*-
import logging
import numpy as np
from ase import units
from ase.build import bulk
from ase.io import Trajectory
import os

# Matlantis / PFP
from matlantis_features.features.md import (
    MDFeature,
    ASEMDSystem,
    VelocityVerletIntegrator,
    LangevinIntegrator,
    NVTBerendsenIntegrator,
    NPTBerendsenIntegrator,
    NPTIntegrator,
)
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode

# --- ユーザー設定項目 (ここを編集してください) ---

# 1. 順番に実行したいIntegratorのリスト
#    選択肢: "NVE", "Langevin", "NVT_Berendsen", "NPT_Berendsen", "NPT""Langevin"VelocityVerletIntegrator
# LangevinIntegrator
# AndersenIntegrator
# NVTBerendsenIntegrator
# NPTBerendsenIntegrator
# NPTIntegrator
INTEGRATORS_TO_RUN = [
    # "NVE",
    # "Langevin"
    "NVT_Berendsen",
    # "NPT_Berendsen",
    # "NPT",
]

# 2. 初期構造のファイルパスを指定
INPUT_STRUCTURE_FILE = "pvdf_opt2.traj"

# 3. MDシミュレーションの共通パラメータ
N_RUN_STEPS = 5000          # 実行するステップ数
TIME_STEP_FS = 1.0         # タイムステップ (fs)
TEMPERATURE_K = 800        # 温度 (K)
PRESSURE_PA = 101325       # 圧力 (Pa) ※NPT系でのみ使用

# 4. ログとトラジェクトリの共通設定
SHOW_PROGRESS = True       # 進捗バーを表示するか
SHOW_LOGGER = True         # ログを標準出力に表示するか
LOGGER_INTERVAL = 50       # ログを出力する間隔 (ステップ数)
TRAJ_FREQ = 50             # トラジェクトリを保存する間隔 (ステップ数)

# 5. PFPモデルの計算設定
PFP_MODEL_VERSION = 'v7.0.0'
PFP_CALC_MODE = EstimatorCalcMode.CRYSTAL_U0

# --- 設定項目ここまで ---


def select_integrator(integrator_type: str):
    """
    指定された文字列に応じて、対応するIntegratorオブジェクトを返す関数。
    """
    # NPT系で使用する圧力 (ASE unitsに変換)
    pressure_au = PRESSURE_PA * units.Pascal

    if integrator_type == "NVE":
        # Microcanonical (NVE) ensemble using the Velocity Verlet algorithm
        return VelocityVerletIntegrator(timestep=TIME_STEP_FS)
        
    elif integrator_type == "Langevin":
        # Langevin dynamics for NVT ensemble
        return LangevinIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            friction=0.002
        )

    elif integrator_type == "NVT_Berendsen":
        # NVT ensemble with Berendsen thermostat
        return NVTBerendsenIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            taut=100.0 * units.fs
        )
        
    elif integrator_type == "NPT_Berendsen":
        # NPT ensemble with Berendsen thermostat and barostat
        return NPTBerendsenIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            pressure=pressure_au,
            taut=100.0 * units.fs,
            taup=1000.0 * units.fs
        )
        
    elif integrator_type == "NPT":
        # NPT ensemble with Nosé-Hoover and Parrinello-Rahman
        return NPTIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            pressure=pressure_au,
            ttime=25.0 * units.fs,
            pfactor=75.0 * units.fs
        )
        
    else:
        raise ValueError(f"エラー: 指定されたIntegrator '{integrator_type}' は不正です。")


def run_md_simulation(initial_atoms, integrator_type, traj_file_name):
    """
    単一のMDシミュレーションを実行する関数。
    """
    print(f"\n{'='*60}")
    print(f" Integrator: '{integrator_type}' のシミュレーションを開始します。")
    print(f" 出力ファイル: '{traj_file_name}'")
    print(f"{'='*60}")

    # PFP計算器の準備
    estimator_fn = pfp_estimator_fn(model_version=PFP_MODEL_VERSION, calc_mode=PFP_CALC_MODE)

    # MDシステムの初期化
    system = ASEMDSystem(initial_atoms)
    
    # 乱数シードを固定して温度を初期化（再現性のため）
    system.init_temperature(temperature=TEMPERATURE_K, stationary=True, rng=np.random.RandomState(seed=12345))
    print(f"システムを {TEMPERATURE_K} K に初期化しました。")

    # Integratorの選択
    try:
        integrator = select_integrator(integrator_type)
    except ValueError as e:
        print(e)
        return

    # MDFeatureの設定
    md = MDFeature(
        integrator=integrator,
        n_run=N_RUN_STEPS,
        show_progress_bar=SHOW_PROGRESS,
        show_logger=SHOW_LOGGER,
        logger_interval=LOGGER_INTERVAL,
        traj_file_name=traj_file_name,
        traj_freq=TRAJ_FREQ,
        traj_props=["energy", "forces", "stress"], # stressを保存
        estimator_fn=estimator_fn,
    )

    # MDシミュレーションの実行
    md_results = md(system)

    print(f"\n--- Integrator: '{integrator_type}' のシミュレーションが完了しました ---")


def main():
    """
    設定に基づいて、複数のMDシミュレーションを逐次実行するメイン関数。
    """
    # Matlantis-featuresのロガー設定
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger("matlantis_features")
    logger.setLevel(logging.INFO)

    # 初期構造の準備
    print("--- 初期構造の準備 ---")
    if INPUT_STRUCTURE_FILE and os.path.exists(INPUT_STRUCTURE_FILE):
        # トラジェクトリファイルの最後のフレームを読み込む
        atoms = Trajectory(INPUT_STRUCTURE_FILE)[-1]
        
        # 真空層を追加し、セルを再設定
        vacuum = 0.5
        positions = atoms.get_positions()
        min_coords = positions.min(axis=0)
        max_coords = positions.max(axis=0)
        cell_lengths = (max_coords - min_coords) + 2 * vacuum
        atoms.set_cell(np.diag(cell_lengths))
        atoms.pbc = True
        
        print(f"構造ファイル '{INPUT_STRUCTURE_FILE}' の最終フレームを読み込み、セルを再設定しました。原子数: {len(atoms)}")
    else:
        print(f"エラー: ファイル '{INPUT_STRUCTURE_FILE}' が見つかりません。")
        return

    # 定義されたIntegratorで順番に計算を実行
    for integrator_name in INTEGRATORS_TO_RUN:
        output_filename = f"md_trajectory_{integrator_name}.traj"
        # 毎回同じ初期構造のコピーを渡す
        run_md_simulation(atoms.copy(), integrator_name, output_filename)
        
    print(f"\n{'='*60}")
    print("全てのシミュレーションが完了しました。")
    print(f"{'='*60}")

if __name__ == "__main__":
    main()


INFO:ga4mp.ga4mp:Sending POST to: https://www.google-analytics.com/mp/collect
INFO:ga4mp.ga4mp:Batch Number: 1
INFO:ga4mp.ga4mp:Status code: 204


--- 初期構造の準備 ---
構造ファイル 'pvdf_opt2.traj' の最終フレームを読み込み、セルを再設定しました。原子数: 601

 Integrator: 'NVT_Berendsen' のシミュレーションを開始します。
 出力ファイル: 'md_trajectory_NVT_Berendsen.traj'
システムを 800 K に初期化しました。


  0%|          | 0/5000 [00:00<?, ?it/s]

INFO:matlantis_features.features.md.md:steps:     0  energy：-4.47 eV/atom  total energy: -4.37 eV/atom  temperature: 803.65 K  volume: 15768 Ang^3  density: 0.668 g/cm^3
INFO:matlantis_features.features.md.md:steps:    50  energy：-4.40 eV/atom  total energy: -4.30 eV/atom  temperature: 773.61 K  volume: 15768 Ang^3  density: 0.668 g/cm^3
INFO:matlantis_features.features.md.md:steps:   100  energy：-4.40 eV/atom  total energy: -4.30 eV/atom  temperature: 823.19 K  volume: 15768 Ang^3  density: 0.668 g/cm^3
INFO:matlantis_features.features.md.md:steps:   150  energy：-4.40 eV/atom  total energy: -4.30 eV/atom  temperature: 764.67 K  volume: 15768 Ang^3  density: 0.668 g/cm^3
INFO:matlantis_features.features.md.md:steps:   200  energy：-4.40 eV/atom  total energy: -4.30 eV/atom  temperature: 788.24 K  volume: 15768 Ang^3  density: 0.668 g/cm^3
INFO:matlantis_features.features.md.md:steps:   250  energy：-4.40 eV/atom  total energy: -4.30 eV/atom  temperature: 777.82 K  volume: 15768 Ang^3  de


--- Integrator: 'NVT_Berendsen' のシミュレーションが完了しました ---

全てのシミュレーションが完了しました。


In [12]:
atoms = Trajectory(INPUT_STRUCTURE_FILE)[-1]
estimator = Estimator(model_version=PFP_MODEL_VERSION,calc_mode=PFP_CALC_MODE)
calculator = ASECalculator(estimator)
atoms.calc=calculator
atoms.get_stress()

PropertyNotImplementedError: stress not present in this calculation

In [18]:
# help(pfp_estimator_fn)
import pfp_api_client
help(pfp_api_client.pfp.estimator.EstimatorCalcMode)

Help on class EstimatorCalcMode in module pfp_api_client.pfp.estimator:

class EstimatorCalcMode(enum.Enum)
 |  EstimatorCalcMode(value, names=None, *, module=None, qualname=None, type=None, start=1, boundary=None)
 |  
 |  Enum class which is used to determine `calc_mode` in `Estimator`.
 |  
 |  :ivar PBE: This mode corresponds to DFT calculations with plane wave basis sets.
 |  :ivar PBE_PLUS_D3: PBE mode with D3 correction enabled.     This mode corresponds to DFT calculations with plane wave basis sets,     plus a D3 dispersion correction.
 |  :ivar PBE_U: PBE mode with Hubbard U parameter. This mode corresponds      to DFT calculations with plane wave basis sets.
 |  :ivar PBE_U_PLUS_D3: PBE_U mode with D3 correction enabled.     This mode corresponds to DFT calculations with plane wave basis sets,     plus a D3 dispersion correction.
 |  :ivar WB97XD: This mode corresponds to DFT calculations with local basis     sets.
 |  
 |  Method resolution order:
 |      EstimatorCalcMode


In [ ]:
atoms=read("pvdf_opt.xyz")


In [2]:
from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import numpy as np
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator,AndersenIntegrator,NPTBerendsenIntegrator,NVTBerendsenIntegrator,VelocityVerletIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.utils.atoms_util import convert_atoms_to_upper
from matlantis_features.atoms import MatlantisAtoms
import numpy as np
import pandas as pd
from pathlib import Path
from ase import units
from ase.build import add_vacuum
import re

In [3]:
import ase
from ase.build import bulk
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution,Stationary
from ase.md.npt import NPT
from ase.md import MDLogger
from ase import units
from time import perf_counter

calc_type = "PFP"
# calc_type = "PFP"

if calc_type == "EMT":
    # ASAP3-EMT calculator
    from asap3 import EMT
    calculator = EMT()
elif calc_type == "PFP":
    # PFP calculator
    from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
    from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
    estimator = Estimator(model_version="v2.0.0",calc_mode=EstimatorCalcMode.CRYSTAL_U0)
    calculator = ASECalculator(estimator)
else:
    raise ValueError(f"Wrong calc_type = {calc_type}!")


# Set up a crystal
# atoms_in = bulk("Cu",cubic=True)
# atoms_in *= 3
# atoms_in.pbc = True
# print("atoms_in = ",atoms_in)

# input parameters
time_step    = 1.0    # fsec
temperature = 1000    # Kelvin
num_md_steps = 20000 
num_interval = 10

sigma   = 1.0     # External pressure in bar
ttime   = 20.0    # Time constant in fs
pfactor = 2e6     # Barostat parameter in GPa
temperature_list = [200,300,400,500,600,700,800,900,1000]

# Print statements
def print_dyn():
    imd = dyn.get_number_of_steps()
    etot  = atoms.get_total_energy()
    temp_K = atoms.get_temperature()
    stress = atoms.get_stress(include_ideal_gas=True)/units.GPa
    stress_ave = (stress[0]+stress[1]+stress[2])/3.0 
    elapsed_time = perf_counter() - start_time
    print(f"  {imd: >3}   {etot:.3f}    {temp_K:.2f}    {stress_ave:.2f}  {stress[0]:.2f}  {stress[1]:.2f}  {stress[2]:.2f}  {stress[3]:.2f}  {stress[4]:.2f}  {stress[5]:.2f}    {elapsed_time:.3f}")

    
# run MD
for i,temperature in enumerate(temperature_list):
    print("i,temperature = ",i,temperature)

    print(f"sigma = {sigma:.1e} bar")
    print(f"ttime = {ttime:.3f} fs")
    print(f"pfactor = {pfactor:.3f} GPa*fs^2")

    temperature_str = str(int(temperature)).zfill(4)
    output_filename = f"pvdf_npt.traj"
    log_filename = output_filename + ".log"
    traj_filename = output_filename + ".traj"
    print("log_filename = ",log_filename)
    print("traj_filename = ",traj_filename)
    
    atoms = read("pvdf_opt.xyz")
    atoms.calc = calculator
    
    # Set the momenta corresponding to T=300K
    MaxwellBoltzmannDistribution(atoms, temperature_K=temperature,force_temp=True)
    Stationary(atoms)
    
    dyn = NPT(atoms,
          time_step*units.fs,
          temperature_K = temperature,
          externalstress = sigma*units.bar,
          ttime = ttime*units.fs,
          pfactor = pfactor*units.GPa*(units.fs**2),
          logfile = log_filename,
          trajectory = traj_filename,
          loginterval=num_interval
          )
    
    print_interval = 1000 if calc_type == "EMT" else num_interval
    dyn.attach(print_dyn, interval=print_interval)
    dyn.attach(MDLogger(dyn, atoms, log_filename, header=True, stress=True, peratom=True, mode="a"), interval=num_interval)

    # Now run the dynamics
    start_time = perf_counter()
    print(f"    imd     Etot(eV)    T(K)    stress(mean,xx,yy,zz,yz,xz,xy)(GPa)  elapsed_time(sec)")
    dyn.run(num_md_steps)

i,temperature =  0 200
sigma = 1.0e+00 bar
ttime = 20.000 fs
pfactor = 2000000.000 GPa*fs^2
log_filename =  pvdf_npt.traj.log
traj_filename =  pvdf_npt.traj.traj
    imd     Etot(eV)    T(K)    stress(mean,xx,yy,zz,yz,xz,xy)(GPa)  elapsed_time(sec)


/home/jovyan/.py311/lib/python3.11/site-packages/ase/md/npt.py:698: RuntimeWarning: divide by zero encountered in scalar divide
  self.pfact = 1.0 / (self.pfactor_given * linalg.det(self._getbox()))


LinAlgError: Singular matrix

In [15]:
# from pfp_estimator.core import pfp_estimator_fn
# from pfp_estimator.estimator import EstimatorCalcMode
# from pfp_estimator.features import MDFeature
# from openmm import LangevinIntegrator

# Assuming `system` is already defined (e.g., from an OpenMM setup)
# and `equil_file` is the desired trajectory file name prefix.
atoms=read("pvdf_opt.xyz")
system=MatlantisAtoms.from_ase_atoms(atoms)
# Define the list of calculation modes you want to run.
calc_modes_to_run = [
    'pbe',
    'pbe_plus_d3',
    'wb97xd',
    'crystal_u0_plus_d3'
]

# Create the integrator
# integrator = LangevinIntegrator(timestep=1.0, temperature=900.0)

# Loop through each calculation mode
for mode_str in calc_modes_to_run:
    print(f"Starting simulation with calc_mode: {mode_str}")

    # Instantiate the estimator function for the current mode
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode=mode_str)
    atoms = atoms_in.copy()
    atoms.calc = calculator
    
    # Set the momenta corresponding to T=300K
    MaxwellBoltzmannDistribution(atoms, temperature_K=temperature,force_temp=True)
    Stationary(atoms)
    
    dyn = NPT(atoms,
          time_step*units.fs,
          temperature_K = temperature,
          externalstress = sigma*units.bar,
          ttime = ttime*units.fs,
          pfactor = pfactor*units.GPa*(units.fs**2),
          logfile = log_filename,
          trajectory = traj_filename,
          loginterval=num_interval
          )
    
    print_interval = 1000 if calc_type == "EMT" else num_interval
    # dyn.attach(print_dyn, interval=print_interval)
    # dyn.attach(MDLogger(dyn, atoms, log_filename, header=True, stress=True, peratom=True, mode="a"), interval=num_interval)

    # Create the MD feature with the new estimator function
    md = MDFeature(
        integrator,
        n_run=4000,
        traj_file_name=f"pvdfcalc_{mode_str}.traj",
        traj_freq=100,
        estimator_fn=estimator_fn
    )

    # Run the MD simulation
    md(system)
    print(f"Simulation with {mode_str} completed.")

Starting simulation with calc_mode: crystal_u0_plus_d3


KeyError: 'total_time'

In [17]:
def set_interface_fixed_atoms_uniform_free(atoms, target_free_thickness=36.0):
    """界面構造で自由領域の厚みを統一する固定原子設定
    
    Parameters:
    -----------
    atoms : ase.Atoms
        対象の原子構造
    target_free_thickness : float
        自由領域の目標厚さ（Å）
        
    Returns:
    --------
    atoms : ase.Atoms
        固定制約が設定された原子構造
    """
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    z_min, z_max = np.min(z_coords), np.max(z_coords)
    total_thickness = z_max - z_min
    
    print(f"自由領域統一固定層設定:")
    print(f"  総材料厚さ: {total_thickness:.2f}Å")
    print(f"  目標自由領域厚さ: {target_free_thickness:.2f}Å")
    
    # 界面情報を取得
    interface_info = identify_interface_region(atoms)
    
    if interface_info['is_interface']:
        print(f"  界面構造検出: 界面位置 = {interface_info['interface_z']:.2f}Å")
        print(f"  Al領域: {interface_info['al_region'][0]:.2f} - {interface_info['al_region'][1]:.2f}Å")
        print(f"  NMC領域: {interface_info['nmc_region'][0]:.2f} - {interface_info['nmc_region'][1]:.2f}Å")
        
        # 界面中心を基準に自由領域を設定
        interface_z = interface_info['interface_z']
        free_half_thickness = target_free_thickness / 2.0
               # 自由領域の範囲
        free_z_min = interface_z - free_half_thickness
        free_z_max = interface_z + free_half_thickness
        
        # 材料範囲内に調整
        free_z_min = max(free_z_min, z_min + 2.0)  # 最低2Å下は固定
        free_z_max = min(free_z_max, z_max - 2.0)  # 最低2Å上は固定
        
        actual_free_thickness = free_z_max - free_z_min
        
        print(f"  自由領域: {free_z_min:.2f} - {free_z_max:.2f}Å")
        print(f"  実際の自由領域厚さ: {actual_free_thickness:.2f}Å")
        
        # 固定原子の選択（自由領域外を固定）
        indices = [
            i for i, z in enumerate(z_coords)
            if z < free_z_min or z > free_z_max
        ]
        
    else:
        # 非界面構造の場合：中心を基準に自由領域を設定
        print(f"  非界面構造: 中心基準で自由領域設定")
        center_z = (z_min + z_max) / 2.0
        free_half_thickness = target_free_thickness / 2.0
        
        free_z_min = max(center_z - free_half_thickness, z_min + 2.0)
        free_z_max = min(center_z + free_half_thickness, z_max - 2.0)
        
        actual_free_thickness = free_z_max - free_z_min
        
        print(f"  中心位置: {center_z:.2f}Å")
        print(f"  自由領域: {free_z_min:.2f} - {free_z_max:.2f}Å")
        print(f"  実際の自由領域厚さ: {actual_free_thickness:.2f}Å")
        
        # 固定原子の選択     
        indices = [
            i for i, z in enumerate(z_coords)
            if z < free_z_min or z > free_z_max
        ]
    
        # 固定制約の設定
        if len(indices) == 0:
            print("  警告: 固定原子がありません")
            return atoms
        
        constraint = FixAtoms(indices=indices)
        atoms.set_constraint(constraint)
        
        # 統計情報
        free_atoms = len(atoms) - len(indices)
        free_ratio = free_atoms / len(atoms)
        fixed_ratio = len(indices) / len(atoms)
        
        print(f"  固定原子数: {len(indices)}/{len(atoms)} ({fixed_ratio*100:.1f}%)")
        print(f"  自由原子数: {free_atoms}/{len(atoms)} ({free_ratio*100:.1f}%)")
        
        # 構造情報を属性として保存
        atoms.info['free_region_thickness'] = actual_free_thickness
        atoms.info['free_region_bounds'] = (free_z_min, free_z_max)
        atoms.info['fixed_atoms_count'] = len(indices)
        atoms.info['free_atoms_count'] = free_atoms
    
    return atoms

In [23]:
system.ase_atoms.get_stress()

PropertyNotImplementedError: stress not present in this calculation

In [ ]:
import ase
from ase.build import bulk
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution,Stationary
from ase.md.npt import NPT
from ase.md import MDLogger
from ase import units
from time import perf_counter

calc_type = "EMT"
# calc_type = "PFP"

if calc_type == "EMT":
    # ASAP3-EMT calculator
    from asap3 import EMT
    calculator = EMT()
elif calc_type == "PFP":
    # PFP calculator
    from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
    from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
    estimator = Estimator(model_version="v2.0.0",calc_mode=EstimatorCalcMode.CRYSTAL_U0)
    calculator = ASECalculator(estimator)
else:
    raise ValueError(f"Wrong calc_type = {calc_type}!")


# Set up a crystal
atoms_in = bulk("Cu",cubic=True)
atoms_in *= 3
atoms_in.pbc = True
print("atoms_in = ",atoms_in)

# input parameters
time_step    = 1.0    # fsec
#temperature = 300    # Kelvin
num_md_steps = 20000 
num_interval = 10

sigma   = 1.0     # External pressure in bar
ttime   = 20.0    # Time constant in fs
pfactor = 2e6     # Barostat parameter in GPa
temperature_list = [200,300,400,500,600,700,800,900,1000]

# Print statements
def print_dyn():
    imd = dyn.get_number_of_steps()
    etot  = atoms.get_total_energy()
    temp_K = atoms.get_temperature()
    stress = atoms.get_stress(include_ideal_gas=True)/units.GPa
    stress_ave = (stress[0]+stress[1]+stress[2])/3.0 
    elapsed_time = perf_counter() - start_time
    print(f"  {imd: >3}   {etot:.3f}    {temp_K:.2f}    {stress_ave:.2f}  {stress[0]:.2f}  {stress[1]:.2f}  {stress[2]:.2f}  {stress[3]:.2f}  {stress[4]:.2f}  {stress[5]:.2f}    {elapsed_time:.3f}")

    
# run MD
for i,temperature in enumerate(temperature_list):
    print("i,temperature = ",i,temperature)

    print(f"sigma = {sigma:.1e} bar")
    print(f"ttime = {ttime:.3f} fs")
    print(f"pfactor = {pfactor:.3f} GPa*fs^2")

    temperature_str = str(int(temperature)).zfill(4)
    output_filename = f"./output/ch6/fcc-Cu_3x3x3_NPT_{calc_type}_{temperature_str}K"
    log_filename = output_filename + ".log"
    traj_filename = output_filename + ".traj"
    print("log_filename = ",log_filename)
    print("traj_filename = ",traj_filename)
    
    atoms = atoms_in.copy()
    atoms.calc = calculator
    
    # Set the momenta corresponding to T=300K
    MaxwellBoltzmannDistribution(atoms, temperature_K=temperature,force_temp=True)
    Stationary(atoms)
    
    dyn = NPT(atoms,
          time_step*units.fs,
          temperature_K = temperature,
          externalstress = sigma*units.bar,
          ttime = ttime*units.fs,
          pfactor = pfactor*units.GPa*(units.fs**2),
          logfile = log_filename,
          trajectory = traj_filename,
          loginterval=num_interval
          )
    
    print_interval = 1000 if calc_type == "EMT" else num_interval
    dyn.attach(print_dyn, interval=print_interval)
    dyn.attach(MDLogger(dyn, atoms, log_filename, header=True, stress=True, peratom=True, mode="a"), interval=num_interval)

    # Now run the dynamics
    start_time = perf_counter()
    print(f"    imd     Etot(eV)    T(K)    stress(mean,xx,yy,zz,yz,xz,xy)(GPa)  elapsed_time(sec)")
    dyn.run(num_md_steps)

In [21]:
# from pfp_estimator.core import pfp_estimator_fn
# from pfp_estimator.estimator import EstimatorCalcMode
# from pfp_estimator.features import MDFeature
# from openmm import LangevinIntegrator

# Assuming `system` is already defined (e.g., from an OpenMM setup)
# and `equil_file` is the desired trajectory file name prefix.
atoms=read("pvdf_opt.xyz")
# system=MatlantisAtoms.from_ase_atoms(atoms)
# Define the list of calculation modes you want to run.
calc_modes_to_run = [
    'pbe',
    'pbe_plus_d3',
    'wb97xd',
    'MOLECULE',
    'crystal',
    'crystal_u0_plus_d3'
]
# set_interface_fixed_atoms_uniform_free(system,)
# Create the integrator
atoms = MatlantisAtoms.from_ase_atoms(atoms)
system = ASEMDSystem(atoms)
integrator = LangevinIntegrator(timestep=1.0, temperature=900.0)

# Loop through each calculation mode
for mode_str in calc_modes_to_run:
    print(f"Starting simulation with calc_mode: {mode_str}")

    # Instantiate the estimator function for the current mode
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode=mode_str)

    # Create the MD feature with the new estimator function
    md = MDFeature(
        integrator,
        n_run=4000,
        traj_file_name=f"pvdfcalc_{mode_str}.traj",
        traj_freq=100,
        estimator_fn=estimator_fn
    )

    # Run the MD simulation
    md(system)
    print(f"Simulation with {mode_str} completed.")

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/pvdfcalc_crystal_u0_plus_d3.traj.
Note: The max disk size of /home/jovyan is about 98G.


Starting simulation with calc_mode: crystal_u0_plus_d3


PropertyNotImplementedError: stress not present in this calculation

In [27]:
import matlantis_features
help(matlantis_features.features.md.MDFeature)

Help on class MDFeature in module matlantis_features.features.md.md:

class MDFeature(matlantis_features.features.base.FeatureBase)
 |  MDFeature(integrator: matlantis_features.features.md.md_integrator_base.MDIntegratorBase, n_run: int, checkpoint_file_name: Optional[str] = None, checkpoint_freq: Optional[int] = 10, traj_file_name: Optional[str] = None, traj_freq: Optional[int] = 1, traj_props: Optional[List[str]] = None, traj_append: bool = False, traj_init_frame: bool = False, show_progress_bar: bool = False, tqdm_options: Optional[Dict[str, Any]] = None, show_logger: bool = False, logger_interval: int = 100, estimator_fn: Optional[Callable[[], pfp_api_client.pfp.estimator.Estimator]] = None)
 |  
 |  The matlantis-feature for molecular dynamics simulation.
 |  
 |  Method resolution order:
 |      MDFeature
 |      matlantis_features.features.base.FeatureBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__ = decorated(self, system: matlantis_features.featur

In [8]:
help(EstimatorCalcMode)

Help on class EstimatorCalcMode in module pfp_api_client.pfp.estimator:

class EstimatorCalcMode(enum.Enum)
 |  EstimatorCalcMode(value, names=None, *, module=None, qualname=None, type=None, start=1, boundary=None)
 |  
 |  Enum class which is used to determine `calc_mode` in `Estimator`.
 |  
 |  :ivar PBE: This mode corresponds to DFT calculations with plane wave basis sets.
 |  :ivar PBE_PLUS_D3: PBE mode with D3 correction enabled.     This mode corresponds to DFT calculations with plane wave basis sets,     plus a D3 dispersion correction.
 |  :ivar PBE_U: PBE mode with Hubbard U parameter. This mode corresponds      to DFT calculations with plane wave basis sets.
 |  :ivar PBE_U_PLUS_D3: PBE_U mode with D3 correction enabled.     This mode corresponds to DFT calculations with plane wave basis sets,     plus a D3 dispersion correction.
 |  :ivar WB97XD: This mode corresponds to DFT calculations with local basis     sets.
 |  
 |  Method resolution order:
 |      EstimatorCalcMode


In [ ]:

integrator = LangevinIntegrator(timestep=1.0, temperature=300.0)
                                md = MDFeature(integrator, n_run=3000, traj_file_name=equil_file, traj_freq=100, estimator_fn=estimator_fn)
                                md(system)